<a href="https://colab.research.google.com/github/CarmenTheodoraCraciun/HairClasification/blob/main/CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importarea biblotecilor si clonarea repo-ului

In [23]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

In [16]:
!rm -rf ./HairClasification
!git clone https://github.com/CarmenTheodoraCraciun/HairClasification.git
!mv ./HairClasification/originalData/Straight/ ./HairClasification/originalData/straight
!mv ./HairClasification/originalData/Wavy/ ./HairClasification/originalData/wavy

Cloning into 'HairClasification'...
remote: Enumerating objects: 6487, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 6487 (delta 16), reused 69 (delta 9), pack-reused 6397 (from 3)
Receiving objects: 100% (6487/6487), 413.76 MiB | 31.33 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Updating files: 100% (2060/2060), done.


# Data processing


In [18]:
def is_image(file_path):
    try:
        img = cv2.imread(file_path)
        return img is not None
    except:
        return False

def resize_image(img, size):
    """
    Resizes an image using bilinear interpolation.

    Args:
        img: Input image.
        size: Desired size of the output image (width, height).

    Returns:
        Resized image.
    """
    original_height, original_width, _ = img.shape
    new_width, new_height = size
    resized_img = np.zeros((new_height, new_width, 3), dtype=np.uint8)

    for i in range(new_width):
        for j in range(new_height):
            #i, j = pixel in the resized image
            # x, y = pixel in the original image
            x = i * (original_width - 1) / (new_width - 1)
            y = j * (original_height - 1) / (new_height - 1)

            # Neighborhood values
            x0 = int(np.floor(x))
            x1 = min(x0 + 1, original_width - 1)
            y0 = int(np.floor(y))
            y1 = min(y0 + 1, original_height - 1)

            # Extract the intensity values ​​of neighbors
            Ia = img[y0, x0] # stanga sus
            Ib = img[y0, x1] # drepata sus
            Ic = img[y1, x0] # stanga jos
            Id = img[y1, x1] # dreapta jos

            # Calculates the weight of each neighboring to the final value
            wa = (x1 - x) * (y1 - y)
            wb = (x - x0) * (y1 - y)
            wc = (x1 - x) * (y - y0)
            wd = (x - x0) * (y - y0)

            # The final value of the new pixel
            pixel = wa * Ia + wb * Ib + wc * Ic + wd * Id
            resized_img[j, i] = np.round(pixel).astype(int)

    return resized_img

def augment_image(img, size=(128, 128), shear_range=0.1, zoom_range=0.1, horizontal_flip=True):
    """
    Preprocesses and augments an image by applying resizing, shearing, zooming, and horizontal flipping.

    Args:
        img: Input image.
        size: Desired size of the output image (width, height).
        shear_range: Range for random shearing.
        zoom_range: Range for random zooming.
        horizontal_flip: Whether to randomly flip the image horizontally.

    Returns:
        Preprocessed and augmented image.
    """
    rows, cols, ch = img.shape
    shear_factor = np.random.uniform(-shear_range, shear_range)
    M_shear = np.array([[1, shear_factor, 0], [0, 1, 0]], dtype=np.float32)
    img = cv2.warpAffine(img, M_shear, (cols, rows))

    zoom_factor = np.random.uniform(1 - zoom_range, 1 + zoom_range)
    img = cv2.resize(img, None, fx=zoom_factor, fy=zoom_factor)

    if zoom_factor < 1: # zoom out
        new_height, new_width = img.shape[:2]
        # Number of pixels added to padding (black) and its addition
        pad_height = (rows - new_height) // 2
        pad_width = (cols - new_width) // 2
        img = cv2.copyMakeBorder(img, pad_height, pad_height,
            pad_width, pad_width, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    else: # zoom in
        # Position of the starting point for cutting
        start_x = (img.shape[1] - cols) // 2
        start_y = (img.shape[0] - rows) // 2
        img = img[start_y:start_y + rows, start_x:start_x + cols]

    if horizontal_flip and np.random.random() < 0.5:
        img = cv2.flip(img, 1)

    img = cv2.resize(img, size)

    return img

def preprocess_images(input_dir, output_dir, size=(128, 128), augment_prob=0.3):
    """
    Resizes images to the specified size and saves them to the output directory.
    Applies augmentations to a random subset of images.

    Args:
        input_dir: Directory containing the input images.
        output_dir: Directory to save the preprocessed images.
        size: Desired size of the output images (width, height).
        augment_prob: Probability of applying augmentation to each image.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("Start processing data.")
    for category in os.listdir(input_dir):
        category_dir = os.path.join(input_dir, category)
        output_category_dir = os.path.join(output_dir, category)
        if not os.path.exists(output_category_dir):
            os.makedirs(output_category_dir)

        num_images = 0
        if os.path.isdir(category_dir):
            for idx, img_name in enumerate(os.listdir(category_dir)):
                img_path = os.path.join(category_dir, img_name)

                if is_image(img_path):
                    img = cv2.imread(img_path)
                    if img is not None:
                        if np.random.rand() < augment_prob:
                            img = augment_image(img, size=size)
                        else:
                            img = resize_image(img, size)
                        new_img_name = f"{category}_{idx}.png"
                        cv2.imwrite(os.path.join(output_category_dir, new_img_name), img)
                        num_images += 1
                    else:
                        print(f"Failed to load image: {img_path}")
                else:
                    print(f"Not an image: {img_path}")

        print(f"Folder {output_category_dir} has {num_images} images.")

    print("The data are processed.")

In [19]:
preprocess_images('./HairClasification/originalData', './HairClasification/processData', size=(128, 128), augment_prob=0.3)

Start processing data.
Folder ./HairClasification/processData/straight has 530 images.
Folder ./HairClasification/processData/dreadlocks has 443 images.
Folder ./HairClasification/processData/wavy has 331 images.
Not an image: ./HairClasification/originalData/curly/rs_1080x1080-200330130638-1080-ariana-grande-curly-hair-instagram-am-033020.gif
Folder ./HairClasification/processData/curly has 515 images.
Folder ./HairClasification/processData/kinky has 232 images.
The data are processed.


##Loading data with labels

In [20]:
def load_images_and_labels(input_dir, size=(128, 128)):
    images = []
    labels = []
    for category in os.listdir(input_dir):
        category_dir = os.path.join(input_dir, category)
        if os.path.isdir(category_dir):
            for img_name in os.listdir(category_dir):
                img_path = os.path.join(category_dir, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, size)
                    images.append(img)
                    labels.append(category)
    return np.array(images), np.array(labels)

In [21]:
input_dir = './HairClasification/processData'
images, labels = load_images_and_labels(input_dir)

##Data Augmentation and Normalization

* Normalization - to contain only 0 and 1 values

In [25]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apply augmentation to all images in the dataset
augmented_images = []
augmented_labels = []

for img, label in zip(images, labels):
    img = img.astype('float32') / 255.0  # Normalize the image
    img = img[np.newaxis, ...]  # Expand dimensions for datagen flow
    gen = datagen.flow(img, batch_size=1)
    # Generate a few augmented images per original image (e.g., 5 augmentations each)
    for _ in range(5):
        aug_img = next(gen)[0]  # Correct usage of next() function
        augmented_images.append(aug_img)
        augmented_labels.append(label)


images_augmented = np.array(augmented_images)
labels_augmented = np.array(augmented_labels)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    images_augmented,
    labels_augmented,
    test_size=0.2,
    random_state=42
)

##One-hot Encoding

* One-hot encoding is a technique used to convert categorical data into a numeric format that a machine learning model can understand
  * e.g. [0,0,0,1,0] means that the hair is straight.


In [27]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

Pregatirea datelor de antrenament si validare

In [28]:
train_datagen = datagen.flow(X_train, y_train_categorical, batch_size=32)
validation_datagen = ImageDataGenerator().flow(X_test, y_test_categorical, batch_size=32)

#Construirea modelului

## Incarcarea in modelul preantrenat VGG16

VGG16 este un model de rețea neurală convoluțională (CNN) binecunoscut și puternic, dezvoltat de echipa de cercetare de la Visual Geometry Group (VGG) de la Universitatea din Oxford. Numele său, VGG16, derivă din structura sa de 16 straturi adânci de rețea neurală.

In [29]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

for layer in base_model.layers:
    layer.trainable = False

##Construirea peste VGG16

In [30]:
model = Sequential()
model.add(base_model)
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

#Compilarea si antrenarea

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_datagen, epochs=20, validation_data=validation_datagen)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
204/257 ━━━━━━━━━━━━━━━━━━━━ 5:37 6s/step - accuracy: 0.4009 - loss: 2.3956

#Evaluarea modelului

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(accuracy) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, accuracy, 'b', label='Accuracy on training data')
plt.plot(epochs, val_accuracy, 'r', label='Accuracy on validation data')
plt.title('Accuracy during training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
loss, accuracy = model.evaluate(X_test_normalized, y_test_categorical)
print(f'Test accuracy: {accuracy:.4f}')

In [ ]:
y_pred_encoded = model.predict(X_test_normalized)
y_pred = np.argmax(y_pred_encoded, axis=1)
y_test = np.argmax(y_test_categorical, axis=1)

print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
conf_df = pd.DataFrame(conf_matrix, index=label_encoder.classes_, columns=label_encoder.classes_)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_df, annot=True, cmap="Blues", fmt="d")
plt.title("Confuzion matrix")
plt.ylabel("Real classes")
plt.xlabel("Predicted classes")
plt.show()